In [1]:
import pandas as pd
import sqlite3

In the cell below we...
- Open up a connection to the SQLite database and store the connection in a variable called `conn`
- Initialize a SQLite cursor object with the variable name `cursor`.

In [2]:
conn = sqlite3.connect('Northwind.sqlite')
cursor = conn.cursor()

In [3]:
table_name_query_df = pd.read_sql("""
SELECT name AS Table_Name FROM sqlite_master WHERE type = 'table';
""", conn)
table_name_query_df

,Table_Name
0,Employee
1,Category
2,Customer
3,Shipper
4,Supplier
5,Order
6,Product
7,OrderDetail
8,CustomerCustomerDemo
9,CustomerDemographic


![Northwind ERD](https://curriculum-content.s3.amazonaws.com/data-science/images/Northwind_ERD.png)

In [8]:
# check the employee table
employee_df = pd.read_sql("""SELECT * FROM Employee;""", conn)
employee_df.head(3)

,Id,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,ReportsTo,PhotoPath
0,1,Davolio,Nancy,Sales Representative,Ms.,1980-12-08,2024-05-01,507 - 20th Ave. E. Apt. 2A,Seattle,North America,98122,USA,(206) 555-9857,5467,None,Education includes a BA in psychology from Col...,2.0,http://accweb/emmployees/davolio.bmp
1,2,Fuller,Andrew,"Vice President, Sales",Dr.,1984-02-19,2024-08-14,908 W. Capital Way,Tacoma,North America,98401,USA,(206) 555-9482,3457,None,Andrew received his BTS commercial in 1974 and...,NaN,http://accweb/emmployees/fuller.bmp
2,3,Leverling,Janet,Sales Representative,Ms.,1995-08-30,2024-04-01,722 Moss Bay Blvd.,Kirkland,North America,98033,USA,(206) 555-3412,3355,None,Janet has a BS degree in chemistry from Boston...,2.0,http://accweb/emmployees/leverling.bmp


In [4]:
# check the customer table
customer_df = pd.read_sql("""SELECT * FROM Customer;""", conn)
customer_df.head(3)

,Id,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,Western Europe,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,Central America,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,Central America,05023,Mexico,(5) 555-3932,None


In [5]:
# check the order table
order_df = pd.read_sql("""SELECT * FROM 'Order';""", conn)
order_df.head(3)

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,10250,HANAR,4,2012-07-08,2012-08-05,2012-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil


In [6]:
# check the order detail table
order_detail_df = pd.read_sql("""SELECT * FROM OrderDetail;""", conn)
order_detail_df.head(3)

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0
2,10248/72,10248,72,34.8,5,0.0


In [24]:
# check the category table
category_df = pd.read_sql("""SELECT * FROM Category;""", conn)
category_df.head(3)

,Id,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"


### Query One: **Find the total sales by customers**

In [7]:
query_1 = """
WITH customer_sales AS (
    SELECT
         c.Id, c.CompanyName,c.ContactName,
         SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS total_sales
    FROM Customer c
    JOIN "Order" o
        ON c.Id = o.CustomerId
    JOIN "OrderDetail" od
        ON o.Id = od.OrderID
    GROUP BY c.Id, c.CompanyName, c.ContactName
)
SELECT *
FROM customer_sales
ORDER BY total_sales DESC;
"""
customer_sales_df = pd.read_sql(query_1, conn)
customer_sales_df.head(10)

,Id,CompanyName,ContactName,total_sales
0,QUICK,QUICK-Stop,Horst Kloss,110277.3050
1,ERNSH,Ernst Handel,Roland Mendel,104874.9785
2,SAVEA,Save-a-lot Markets,Jose Pavarotti,104361.9500
3,RATTC,Rattlesnake Canyon Grocery,Paula Wilson,51097.8005
4,HUNGO,Hungry Owl All-Night Grocers,Patricia McKenna,49979.9050
5,HANAR,Hanari Carnes,Mario Pontes,32841.3700
6,KOENE,Königlich Essen,Philip Cramer,30908.3840
7,FOLKO,Folk och fä HB,Maria Larsson,29567.5625
8,MEREP,Mère Paillarde,Jean Fresnière,28872.1900
9,WHITC,White Clover Markets,Karl Jablonski,27363.6050


### Query Two: **Find the total sales by each employee**

In [25]:
query_2 = """
WITH employee_sales AS (
     SELECT
         CONCAT(e.LastName, ' ', e.LastName) AS employee_name, e.Title,
         SUM(od.UnitPrice * od.Quantity * (1- od.Discount)) AS employee_total_sales
     FROM Employee e
     JOIN "Order" o
         ON e.Id = o.EmployeeId
     JOIN "OrderDetail" od
         ON o.Id = od.OrderId
     GROUP BY employee_name
)
SELECT *
FROM employee_sales
ORDER BY employee_total_sales DESC;
"""
employee_sales_df = pd.read_sql(query_2, conn)
employee_sales_df.head(10)

,employee_name,Title,employee_total_sales
0,Peacock Peacock,Sales Representative,232890.8460
1,Leverling Leverling,Sales Representative,202812.8430
2,Davolio Davolio,Sales Representative,192107.6045
3,Fuller Fuller,"Vice President, Sales",166537.7550
4,Callahan Callahan,Inside Sales Coordinator,126862.2775
5,King King,Sales Representative,124568.2350
6,Dodsworth Dodsworth,Sales Representative,77308.0665
7,Suyama Suyama,Sales Representative,73913.1295
8,Buchanan Buchanan,Sales Manager,68792.2825


### Query 3: **Find the total Revenue by products**

In [11]:
employee_df.head()

,Id,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,ReportsTo,PhotoPath
0,1,Davolio,Nancy,Sales Representative,Ms.,1980-12-08,2024-05-01,507 - 20th Ave. E. Apt. 2A,Seattle,North America,98122,USA,(206) 555-9857,5467,None,Education includes a BA in psychology from Col...,2.0,http://accweb/emmployees/davolio.bmp
1,2,Fuller,Andrew,"Vice President, Sales",Dr.,1984-02-19,2024-08-14,908 W. Capital Way,Tacoma,North America,98401,USA,(206) 555-9482,3457,None,Andrew received his BTS commercial in 1974 and...,NaN,http://accweb/emmployees/fuller.bmp
2,3,Leverling,Janet,Sales Representative,Ms.,1995-08-30,2024-04-01,722 Moss Bay Blvd.,Kirkland,North America,98033,USA,(206) 555-3412,3355,None,Janet has a BS degree in chemistry from Boston...,2.0,http://accweb/emmployees/leverling.bmp
3,4,Peacock,Margaret,Sales Representative,Mrs.,1969-09-19,2025-05-03,4110 Old Redmond Rd.,Redmond,North America,98052,USA,(206) 555-8122,5176,None,Margaret holds a BA in English literature from...,2.0,http://accweb/emmployees/peacock.bmp
4,5,Buchanan,Steven,Sales Manager,Mr.,1987-03-04,2025-10-17,14 Garrett Hill,London,British Isles,SW1 8JR,UK,(71) 555-4848,3453,None,Steven Buchanan graduated from St. Andrews Uni...,2.0,http://accweb/emmployees/buchanan.bmp


In [15]:
product_df = pd.read_sql("""SELECT * FROM Product;""", conn)
product_df.head(3)

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.0,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.0,13,70,25,0


In [ ]:
query_2 = """
WITH employee_sales AS (
     SELECT
         CONCAT(e.LastName, ' ', e.LastName) AS employee_name, e.Title,
         SUM(od.UnitPrice * od.Quantity * (1- od.Discount)) AS employee_total_sales
     FROM Employee e
     JOIN "Order" o
         ON e.Id = o.EmployeeId
     JOIN "OrderDetail" od
         ON o.Id = od.OrderId
     GROUP BY employee_name
)
SELECT *
FROM employee_sales
ORDER BY employee_total_sales DESC;
"""
employee_sales_df = pd.read_sql(query_2, conn)
employee_sales_df.head(10)

In [16]:
product_df.head()

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


In [19]:
# join the products with order_detail
query_3 = """
WITH revenue_per_product AS (
    SELECT
         p.Id, p.ProductName,
         SUM(od.UnitPrice * od.Quantity * (1 - Discount)) AS product_revenue
    FROM Product p
    JOIN "OrderDetail" od
          ON p.Id = od.ProductId
    GROUP BY p.ProductName, p.Id
)
SELECT *
FROM revenue_per_product
ORDER BY product_revenue DESC;
"""
product_revenue_df = pd.read_sql(query_3, conn)
product_revenue_df.head(10)

,Id,ProductName,product_revenue
0,38,Côte de Blaye,141396.735
1,29,Thüringer Rostbratwurst,80368.672
2,59,Raclette Courdavault,71155.700
3,62,Tarte au sucre,47234.970
4,60,Camembert Pierrot,46825.480
5,56,Gnocchi di nonna Alice,42593.060
6,51,Manjimup Dried Apples,41819.650
7,17,Alice Mutton,32698.380
8,18,Carnarvon Tigers,29171.875
9,28,Rössle Sauerkraut,25696.640


### Query 4: **Find the Best selling product by Quantity**

In [21]:
query_4 = """
WITH best_selling_product AS (
    SELECT p.ProductName,
            SUM(od.Quantity) AS total_quantity_sold
    FROM Product p
    JOIN "OrderDetail" od
          ON p.Id = od.ProductId
    GROUP BY p.ProductName
)
SELECT *
FROM best_selling_product
ORDER BY total_quantity_sold DESC;
"""
best_selling_product_df = pd.read_sql(query_4, conn)
best_selling_product_df.head(10)

,ProductName,total_quantity_sold
0,Camembert Pierrot,1577
1,Raclette Courdavault,1496
2,Gorgonzola Telino,1397
3,Gnocchi di nonna Alice,1263
4,Pavlova,1158
5,Rhönbräu Klosterbier,1155
6,Guaraná Fantástica,1125
7,Boston Crab Meat,1103
8,Tarte au sucre,1083
9,Chang,1057


### Query 5: **Find the Sales by Category**

,Id,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"


In [18]:
query_5 = """
WITH sales_by_category AS (
    SELECT
        cg.CategoryName,
        SUM(od.UnitPrice * od.Quantity * (1-od.Discount)) AS total_sales_category
    FROM Category cg
    JOIN "OrderDetail" od
          ON cg.Id = od.ProductId
    GROUP BY cg.CategoryName
)
SELECT *
FROM sales_by_category
ORDER BY total_sales_category DESC;
"""
sales_category_df = pd.read_sql(query_5, conn)
sales_category_df.head(10)

,CategoryName,total_sales_category
0,Produce,22044.30
1,Condiments,16355.96
2,Beverages,12788.10
3,Seafood,12772.00
4,Dairy Products,8567.90
5,Meat/Poultry,7137.00
6,Grains/Cereals,5347.20
7,Confections,3044.00


### Query 6: **Find the sales trend for each month**

In [19]:
order_df.head(2)

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany


In [20]:
order_detail_df.head()

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0
2,10248/72,10248,72,34.8,5,0.0
3,10249/14,10249,14,18.6,9,0.0
4,10249/51,10249,51,42.4,40,0.0


In [21]:
# Here we establish the month and year with the highest revenue
query_6 = """
WITH monthly_sales AS (
   SELECT
       strftime('%Y-%m', o.OrderDate) AS ordered_month,
       SUM(od.UnitPrice * od.Quantity * (1-od.Discount)) AS monthly_revenue
   FROM "Order" o
   JOIN "OrderDetail" od
        ON  o.Id = od.OrderId
   GROUP BY ordered_month
)
SELECT *
FROM monthly_sales
ORDER BY monthly_revenue DESC;
"""
monthly_trends_df = pd.read_sql(query_6, conn)
monthly_trends_df.head(10)
   #strftime('%Y-%m', o.OrderDate) AS order_month,

,ordered_month,monthly_revenue
0,2014-04,123798.6825
1,2014-03,104854.1550
2,2014-02,99415.2875
3,2014-01,94222.1105
4,2013-12,71398.4285
5,2013-10,66749.2260
6,2013-01,61258.0700
7,2013-09,55629.2425
8,2013-05,53781.2900
9,2013-04,53032.9525


In [22]:
# Here find the sales trend for each month
query_6 = """
WITH monthly_sales AS (
   SELECT
       strftime('%Y-%m', o.OrderDate) AS ordered_month,
       SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS monthly_revenue
   FROM "Order" o
   JOIN "OrderDetail" od
        ON o.Id = od.OrderId
   GROUP BY ordered_month
)
SELECT *
FROM monthly_sales
ORDER BY ordered_month DESC;
"""
monthly_trends_df = pd.read_sql(query_6, conn)
monthly_trends_df.head(10)

,ordered_month,monthly_revenue
0,2014-05,18333.6305
1,2014-04,123798.6825
2,2014-03,104854.1550
3,2014-02,99415.2875
4,2014-01,94222.1105
5,2013-12,71398.4285
6,2013-11,43533.8090
7,2013-10,66749.2260
8,2013-09,55629.2425
9,2013-08,47287.6700


In [23]:
# To get the monthly revenue for the sales
query_6_sales = """
WITH monthly_sales AS (
   SELECT
       strftime("%m", o.OrderDate) AS ordered_month,
       SUM(od.UnitPrice * od.Quantity * (1-od.Discount)) AS monthly_revenue
    FROM "Order" o
    JOIN "OrderDetail" od
        ON  o.Id = od.OrderId
    GROUP BY ordered_month
)
SELECT *
FROM monthly_sales
ORDER BY monthly_revenue DESC;
"""
monthly_trends_df = pd.read_sql(query_6_sales, conn)
monthly_trends_df.head(10)

,ordered_month,monthly_revenue
0,04,176831.6350
1,01,155480.1805
2,03,143401.3750
3,02,137898.9225
4,12,116638.0585
5,10,104264.9510
6,11,89133.8540
7,09,82010.6425
8,07,78882.7525
9,08,72772.9450


### Query 7: **Find the customers with more than 5 orders**

In [24]:
customer_df.head(3)

,Id,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,Western Europe,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,Central America,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,Central America,05023,Mexico,(5) 555-3932,None


In [25]:
order_df.head(3)

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,10250,HANAR,4,2012-07-08,2012-08-05,2012-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil


In [26]:
query_7 = """
WITH customer_more_5_orders AS (
   SELECT
       o.CustomerId, c.ContactName,
       COUNT(CustomerId) AS order_count_per_customer
    FROM "Order" o
    JOIN Customer c
        ON o.CustomerId = c.Id
    GROUP BY o.CustomerId, c.ContactName
    HAVING COUNT(CustomerId) > 5
)
SELECT *
FROM customer_more_5_orders
ORDER BY order_count_per_customer
;
"""
customer_more_5_orders_df = pd.read_sql(query_7, conn)
customer_more_5_orders_df.head()

,CustomerId,ContactName,order_count_per_customer
0,ALFKI,Maria Anders,6
1,CACTU,Patricio Simpson,6
2,DRACD,Sven Ottlieb,6
3,FRANS,Paolo Accorti,6
4,PERIC,Guillermo Fernández,6


In [27]:
# To check for any duplicates on the dataframe
customer_more_5_orders_df.duplicated().any()
# There are no duplicates for the customers with more than 5 orders

np.False_

### Query 8: **Find the Customers with the highest revenue**

In [28]:
customer_df.head(3)

,Id,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,Western Europe,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,Central America,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,Central America,05023,Mexico,(5) 555-3932,None


In [29]:
order_detail_df.head(2)

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0


In [30]:
order_df.head(2)

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany


In [31]:
product_df.head(2)

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.0,17,40,25,0


In [32]:
query_8 = """
WITH customers_highest_revenue AS (
  SELECT
      c.Id, c.ContactName,
      SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS customer_revenue
  FROM Customer c
  JOIN "Order" o
    ON c.Id = o.CustomerId
  JOIN "OrderDetail" od
    ON o.Id = od.OrderId
  GROUP BY c.Id, c.ContactName
)
SELECT *
FROM customers_highest_revenue
ORDER BY customer_revenue DESC
LIMIT 10
;
"""
customers_highest_revenue_df = pd.read_sql(query_8, conn)
customers_highest_revenue_df

,Id,ContactName,customer_revenue
0,QUICK,Horst Kloss,110277.3050
1,ERNSH,Roland Mendel,104874.9785
2,SAVEA,Jose Pavarotti,104361.9500
3,RATTC,Paula Wilson,51097.8005
4,HUNGO,Patricia McKenna,49979.9050
5,HANAR,Mario Pontes,32841.3700
6,KOENE,Philip Cramer,30908.3840
7,FOLKO,Maria Larsson,29567.5625
8,MEREP,Jean Fresnière,28872.1900
9,WHITC,Karl Jablonski,27363.6050


### Query 9: **Determine the average order value**

In [33]:
order_df.head(2)

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany


In [34]:
order_detail_df.head(2)

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0


In [35]:
# query_8 = """
# WITH average_order_value AS (
#    SELECT o.CustomerId,
#           SUM(od.UnitPrice * od.Quantity * (1-od.Discount)) AS value_each_order
#     FROM "Order" o
#     JOIN "OrderDetail" od
#         ON o.Id = od.OrderId
#     GROUP BY o.CustomerId
# )
# SELECT *
# FROM average_order_value
# ORDER BY value_each_order DESC;
# """
# average_order_value_df = pd.read_sql(query_8, conn)
# average_order_value_df.head()

In [36]:
query_8 = """
WITH order_value AS (
   SELECT o.Id AS order_id,
          o.CustomerId,
          SUM(od.UnitPrice * od.Quantity * (1-od.Discount)) AS order_value
    FROM "Order" o
    JOIN "OrderDetail" od
        ON o.Id = od.OrderId
    GROUP BY o.CustomerId, o.Id
),
average_order_value AS (
   SELECT
       CustomerId,
       AVG(order_value) AS av_order_value
    FROM order_value
    GROUP BY CustomerId
)
SELECT *
FROM average_order_value
ORDER BY av_order_value DESC
;
"""
average_order_value_df = pd.read_sql(query_8, conn)
average_order_value_df.head()

,CustomerId,av_order_value
0,QUICK,3938.475179
1,ERNSH,3495.832617
2,SAVEA,3366.514516
3,RATTC,2838.766694
4,HUNGO,2630.521316


### Query 10: **Find the product that have never been sold**

In [41]:
product_df.head(3)

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.0,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.0,13,70,25,0


In [40]:
order_detail_df.head(3)

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0
2,10248/72,10248,72,34.8,5,0.0


In [36]:
query_10 = """
WITH product_not_sold AS (
    SELECT DISTINCT P.
        p.ProductName,




)

"""

In [18]:
product_df.head()
# find the total revenue generated by each employee on the various product sold
query_emp_revenue = """
                    WITH employee_sales_product AS (SELECT CONCAT(e.FirstName, ' ', e.LastName)                AS employee_name,
                                                           e.Title,
                                                           p.ProductName,
                                                           SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS employee_product_sales_revenue
                                                    FROM Product p
                                                             JOIN Employee e
                                                                  ON p.Id = e.Id
                                                             JOIN "Order" o
                                                                  ON e.Id = o.EmployeeId
                                                             JOIN "OrderDetail" od
                                                                  ON o.Id = od.OrderId
                                                    GROUP BY p.ProductName, employee_name)
                    SELECT *
                    FROM employee_sales_product
                    ORDER BY employee_product_sales_revenue DESC; \
                    """
employee_product_df = pd.read_sql(query_emp_revenue, conn)
employee_product_df.head()

,employee_name,Title,ProductName,employee_product_sales_revenue
0,Margaret Peacock,Sales Representative,Chef Anton's Cajun Seasoning,232890.8460
1,Janet Leverling,Sales Representative,Aniseed Syrup,202812.8430
2,Nancy Davolio,Sales Representative,Chai,192107.6045
3,Andrew Fuller,"Vice President, Sales",Chang,166537.7550
4,Laura Callahan,Inside Sales Coordinator,Northwoods Cranberry Sauce,126862.2775
